# 1
Задан простой класс Fraction для представления дробей:


    class Fraction(object):
        def __init__(self, num, den):
            self.__num = num
            self.__den = den
            self.reduce()

        def __str__(self):
            return "%d/%d" % (self.__num, self.__den)

        def reduce(self):
            g = Fraction.gcd(self.__num, self.__den)
            self.__num /= g
            self.__den /= g

        @staticmethod
        def gcd(n, m):
            if m == 0:
                return n
            else:
                return Fraction.gcd(m, n % m)
Дополнить класс таким образом, чтобы выполнялся следующий код:


    frac = Fraction(7, 2)
    print(-frac) # выводит -7/2
    print(~frac) # выводит 2/7
    print(frac**2) # выводит 49/4
    print(float(frac)) # выводит 3.5
    print(int(frac)) # выводит 3



In [1]:
class Fraction(object):

    def __init__(self, num, den):
        self.__num = num
        self.__den = den
        self.reduce()

    def __str__(self):
        return "%d/%d" % (self.__num, self.__den)

    def reduce(self):
        g = Fraction.gcd(self.__num, self.__den)
        self.__num /= g
        self.__den /= g

    @staticmethod
    def gcd(n, m):
        if m == 0:
            return n
        else:
            return Fraction.gcd(m, n % m)

    def __int__(self):
        return int(self.__num/self.__den)

    def __float__(self):
        return (self.__num/self.__den)

    def __invert__(self):
        return Fraction(self.__den, self.__num)

    def __neg__(self):
        return Fraction(-self.__num, self.__den)

    def __pow__(self, power):
         return Fraction(self.__num**power, self.__den**power)
            
frac = Fraction(7, 2)
print(-frac) # выводит -7/2
print(~frac) # выводит 2/7
print(frac**2) # выводит 49/4
print(float(frac)) # выводит 3.5
print(int(frac)) # выводит 3

-7/2
2/7
49/4
3.5
3


## 2
Напишите классы «Книга» (с обязательными полями: название, автор,
код), «Библиотека» (с обязательными полями: адрес, номер) и
корректно свяжите их. Код книги должен назначаться автоматически
при добавлении книги в библиотеку (используйте для этого
статический член класса). Если в конструкторе книги указывается в
параметре пустое название, необходимо сгенерировать исключение
(например, ValueError). Книга должна реализовывать интерфейс
Taggable с методом tag(), который создает на основе строки набор тегов
(разбивает строку на слова и возвращает только те, которые
начинаются с большой буквы). Например, tag() для книги с названием
‘War and Peace’ вернет список тегов `[‘War’, ‘Peace’]`. Реализуйте классы
таким образом, чтобы корректно выполнялся следующий код:

    lib = Library(1, ’51 Some str., NY’)
    lib += Book(‘Leo Tolstoi’, ‘War and Peace’)
    lib += Book(‘Charles Dickens’, ‘David Copperfield’)
    for book in lib:
    # вывод в виде: [1] L.Tolstoi ‘War and Peace’
    print(book)
    # вывод в виде: [‘War’, ‘Peace’]
    print(book.tag())


In [22]:
class Taggable(object):
    def tag(self):
        pass

class Book(Taggable):
    id = 0
    def __init__(self, author, title):
        if title == "":
             raise ValueError("incorrect title")
        self.title = title
        self.author = author

    def tag(self):
        self.tags = list()
        for w in self.title.split(' '):
            if w.istitle():
                self.tags.append(w)
        return self.tags
    def __str__(self):
        return f"[{self.id}] {self.author} \'{self.title}\'"

       


class Library:  
    def __init__(self, id, address):
        self.address = address
        self.id = id
        self.books  = list()
        self.itr = 0

    def __iadd__(self, book):
        book.id = len(self.books)+1
        self.books.append(book)
        return self

    def __iter__(self):
        return self

    def __next__(self):
        if self.itr < len(self.books):
            self.itr +=1
            return self.books[self.itr-1]
        raise StopIteration

lib = Library(1, '51 Some str., NY')
lib += Book('Leo Tolstoi', 'War and Peace')
lib += Book('Charles Dickens', 'David Copperfield')
for book in lib:
    # вывод в виде: [1] L.Tolstoi ‘War and Peace’
    print(book)
    # вывод в виде: [‘War’, ‘Peace’]
    print(book.tag())

[1] Leo Tolstoi 'War and Peace'
['War', 'Peace']
[2] Charles Dickens 'David Copperfield'
['David', 'Copperfield']


## 3

Создайте графическую оболочку для скрипта, написанного в ходе
выполнения задания № 4 лабораторной работы № 2, в виде диалогового
окна (рис. 2). Рекомендуется использовать wxPython или PyQt.


In [46]:
import re
import PySimpleGUI as sg
from copy import deepcopy
from datetime import datetime

sg.theme('Material2')
font = ('Segoe UI', 10)
DEBUG = True
if DEBUG:
    path = "./task-four.txt"
else:
    path = input('Введите путь к файлу: ')

pattern = re.compile(r'\d{2}-\d{2}-\d{4}')
text = ""
layout = [
    [sg.FileBrowse(file_types= [("Text Files", "*.txt")] ,key="-IN-", font=font, enable_events=True)],
    [sg.Text(text,  font=font, key="text", size=(70, 50))]
    ]
window = sg.Window('Window Title', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event=="Exit":
        break
    if event == '-IN-':
        if text != "":
            text+="\n"
        text+=f"Файл {values['-IN-']} был обработан {datetime.now().strftime('%m.%d.%Y %H:%M:%S')}: \n\n"
        file =  open(values['-IN-'], "r", encoding="utf-8")
        for i, line in enumerate(file.readlines()):
            result = pattern.search(line)
            if result != None:
                text+=(f'Строка {i}, позиция {result.start()} : найдено \'{result.group()}\'\n')
        window["text"].update(text)

window.close()
    

*(давайте закроем глаза на то что логов нема, я слишком долго это делал потому что хотел похайповее интерфейс и сойду с ума если буду еще чо то тут менять)*

## 4
Напишите простой класс StringFormatter для форматирования строк со
следующим функционалом:

* удаление всех слов из строки, длина которых меньше n букв;

* замена всех цифр в строке на знак «*»;

* вставка по одному пробелу между всеми символами в строке;

* сортировка слов по размеру;

* сортировка слов в лексикографическом порядке.

Примечание. Разделители слов можно задавать отдельно. По
умолчанию в качестве разделителя принимается только символ
пробела.


In [75]:
#вообще это бы нврн модулем делать
class StringFormatter():
    def __init__(self, string, delimeter = " "):
        self.string = string
        self.delimeter = delimeter
    def delete_long_words(self, n):
        new_string = ""
        for w in self.string.split(self.delimeter):
            if len(w)>=n:
                new_string+=f"{w}{self.delimeter}"
        self.string = new_string[:-1]
        return self.string 
    def obscure_digits(self):
        new_string = ""
        for c in self.string:
            new_string+= '*' if c.isdigit() else c
        self.string = new_string
        return self.string 
    def add_spaces(self):
        new_string = ""
        for c in self.string:
            new_string+=f" {c}"
        self.string = new_string
        return self.string 
    def sort_words(self, ascending = True):
        words = self.string.split(self.delimeter)
        words.sort(key = lambda word: len(word))
        if not ascending: words.reverse()
        self.string = self.delimeter.join(words)
        return self.string
    def lex(self, str):
        sum = 0
        for i,c in enumerate(str):
            sum+=ord(c.lower())/10**i
        return sum
    def sort_words_lex(self, ascending = True):
        words = self.string.split(self.delimeter)
        words.sort(key = self.lex)
        if not ascending: words.reverse()
        self.string = self.delimeter.join(words)
        return self.string
    def get_string(self):
        return self.string

## 5
Напишите скрипт с графическим интерфейсом пользователя для
демонстрации работы класса StringFormatter. Примеры окон приведены
на рис. 4 (все элементы управления необходимо обязательно
реализовать те же, что присутствуют на рисунке). Разные комбинации
отмеченных чекбоксов приводят к разным цепочкам операций
форматирования задаваемой в верхнем поле строки с разными
результатами:

In [76]:
import PySimpleGUI as sg

sg.theme('Material2')
font = ('Segoe UI', 10)

layout = [
    [sg.Text("Строка:\t",  font=font), sg.Input(key='_INPUT_', background_color="white", text_color="black")],
    [sg.Checkbox('Удалять слова размера меньше', key="_DEL_"), sg.Spin(values=[i for i in range(1, 1000)], background_color="white", text_color="black", key="_N_"), sg.Text("букв", font=font)],
    [sg.Checkbox('Заменить все цифры на *', key="_REPL_")],
    [sg.Checkbox('Вставить по пробелу между символами', key="_SPACE_")],
    [sg.Checkbox('Сортировать слова в строке', key="_SORT_")],
    [sg.Radio('По размеру', "g", font=font, default=True, key="_LEN_")], [sg.Radio('Лексиграфически',"g", font=font,key="_LEX_")],
    [sg.Button('Форматировать', key="_FORMAT_")],
    [sg.Text("Результат:\t",  font=font), sg.Input(key='_OUTPUT_', background_color="white", text_color="black")]
    ]

window = sg.Window('Window Title', layout, size=(500, 300))
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event=="Exit":
        break
    if event == '_FORMAT_':
        if values['_INPUT_'] != "":
            sf = StringFormatter(values['_INPUT_'])
            if  values['_DEL_']:
                sf.delete_long_words(values["_N_"])
            if values['_REPL_']:
                sf.obscure_digits()
            if values['_SPACE_']:
                sf.add_spaces()
            if values['_SORT_']:
                if values['_LEN_']:
                    sf.sort_words()
                else:
                    sf.sort_words_lex()     
            window["_OUTPUT_"].update(sf.get_string())

window.close()